In [1]:
import pandas as pd

import numpy as np


In [2]:
expid = 132
predDf = pd.read_csv(
    "/home/alextay96/Desktop/all_workspace/new_workspace/DLDataPipeline/data/case_study/case_study_15/case_study_pred.csv"
)
gtDf = pd.read_csv(
    "/home/alextay96/Desktop/all_workspace/new_workspace/DLDataPipeline/data/case_study/case_study_15/case_study_gt.csv"
)

visionPredDf1 = pd.read_csv(f"/home/alextay96/Desktop/all_workspace/new_workspace/DLDataPipeline/data/results/{expid}/Saloon - 4 Dr/Saloon - 4 Dr_imgs_pred_output.csv")
visionPredDf2 = pd.read_csv(f"/home/alextay96/Desktop/all_workspace/new_workspace/DLDataPipeline/data/results/{expid}/Hatchback - 5 Dr/Hatchback - 5 Dr_imgs_pred_output.csv")
visionPredDf3 = pd.read_csv(f"/home/alextay96/Desktop/all_workspace/new_workspace/DLDataPipeline/data/results/{expid}/SUV - 5 Dr/SUV - 5 Dr_imgs_pred_output.csv")
visionPredDf = pd.concat([visionPredDf1, visionPredDf2,visionPredDf3 ])
majorParts = [x.replace("vision_", "") for x in  visionPredDf.columns]
majorParts

['Unnamed: 0',
 'CaseID',
 'bonnet',
 'bumper_front',
 'engine',
 'fender_front_lh',
 'front_panel',
 'grille',
 'headlamp_lh',
 'windscreen_front',
 'bumper_rear',
 'rear_compartment',
 'rear_panel',
 'rear_quarter_rh',
 'tail_lamp_rh',
 'windscreen_rear',
 'rear_quarter_lh',
 'tail_lamp_lh',
 'fender_front_rh',
 'headlamp_rh']

In [3]:
len(predDf)

29

In [4]:
targetCols = [x for x in predDf.columns if "0" not in x and x != "CaseID"]
allRows = []
for c in targetCols:
    for caseId in predDf["CaseID"].unique().tolist():
        visionPred = -1
        if(c in majorParts):
            visionPred = visionPredDf[visionPredDf["CaseID"] == caseId][f"vision_{c}"].item()
            

        pred = predDf[predDf["CaseID"] == caseId][c].item()
        gt = gtDf[gtDf["CaseID"] == caseId][c].item()
        fp = 1 if gt == 0 and pred == 1 else 0
        fn = 1 if gt == 1 and pred == 0 else 0
        allRows.append(
            {
                "CaseID" : caseId,
                "part" : c,
                "v_pred":visionPred,
                "pred" : pred,
                "gt" : gt,
                "fp" : fp, 
                "fn" : fn,
                "inspect" : None,
                "remarks" : None

            }
        )
caseStudyInspectDf = pd.json_normalize(allRows)
caseStudyInspectDf.sort_values(by = ["CaseID", "part"], inplace=True)

In [5]:
caseStudyInspectDf

,CaseID,part,v_pred,pred,gt,fp,fn,inspect,remarks
0,10015204,bonnet,0,0,0,0,0,None,None
29,10015204,bumper_front,0,0,0,0,0,None,None
145,10015204,bumper_rear,1,1,1,0,0,None,None
174,10015204,engine,0,0,0,0,0,None,None
232,10015204,fender_front_lh,0,0,0,0,0,None,None
...,...,...,...,...,...,...,...,...,...
492,13445707,rear_quarter_rh,0,0,0,0,0,None,None
347,13445707,tail_lamp_lh,0,0,0,0,0,None,None
376,13445707,tail_lamp_rh,0,0,1,0,1,None,None
405,13445707,windscreen_front,0,0,0,0,0,None,None


In [6]:
len(caseStudyInspectDf["part"].unique())

18

In [7]:
majorParts = 25 * 30
majorParts

750

In [8]:
154 / majorParts

0.20533333333333334

In [9]:
caseStudyWrong = caseStudyInspectDf[(caseStudyInspectDf["pred"] !=caseStudyInspectDf["gt"] )
                                    & (caseStudyInspectDf["v_pred"] != -1)
                                    
                                    ]
caseStudyWrong

,CaseID,part,v_pred,pred,gt,fp,fn,inspect,remarks
406,10015204,rear_compartment,1,1,0,1,0,None,None
290,10015204,rear_quarter_lh,1,1,0,1,0,None,None
464,10015204,rear_quarter_rh,1,1,0,1,0,None,None
348,10015204,tail_lamp_rh,1,1,0,1,0,None,None
175,10035201,engine,1,1,0,1,0,None,None
...,...,...,...,...,...,...,...,...,...
56,13324832,bumper_front,1,1,0,1,0,None,None
85,13324832,grille,1,1,0,1,0,None,None
260,13445707,fender_front_lh,0,0,1,0,1,None,None
289,13445707,fender_front_rh,0,1,0,1,0,None,None


In [10]:
len(caseStudyWrong) / len(caseStudyInspectDf)

0.1532567049808429

In [11]:

caseStudyWrongVis = caseStudyInspectDf[(caseStudyInspectDf["pred"] !=caseStudyInspectDf["v_pred"]) & (caseStudyInspectDf["v_pred"] != -1) ]
caseStudyWrongVis

,CaseID,part,v_pred,pred,gt,fp,fn,inspect,remarks
493,10015204,windscreen_rear,1,0,0,0,0,None,None
177,10245081,engine,0,1,1,0,0,None,None
322,10245081,tail_lamp_lh,0,1,0,1,0,None,None
496,10245081,windscreen_rear,1,0,0,0,0,None,None
236,10273312,fender_front_lh,1,0,0,0,0,None,None
294,10273312,rear_quarter_lh,0,1,0,1,0,None,None
5,10277513,bonnet,0,1,0,1,0,None,None
36,10529294,bumper_front,0,1,1,0,0,None,None
152,10529294,bumper_rear,1,0,0,0,0,None,None
65,10529294,grille,0,1,1,0,0,None,None


In [12]:
# majorPartDf = caseStudyInspectDf[caseStudyInspectDf["v_pred"] != - 1]
majorPartDf = caseStudyInspectDf

In [13]:
majorPartDf["part"].unique()

array(['bonnet', 'bumper_front', 'bumper_rear', 'engine',
       'fender_front_lh', 'fender_front_rh', 'front_panel', 'grille',
       'headlamp_lh', 'headlamp_rh', 'rear_compartment', 'rear_panel',
       'rear_quarter_lh', 'rear_quarter_rh', 'tail_lamp_lh',
       'tail_lamp_rh', 'windscreen_front', 'windscreen_rear'],
      dtype=object)

In [14]:
fpr = len(majorPartDf[(majorPartDf["pred"]  == 1) & (majorPartDf["gt"]  == 0) ]) / len(majorPartDf[ (majorPartDf["gt"]  == 0)])
fpr

0.11083743842364532

In [15]:
fnr = len(majorPartDf[(majorPartDf["pred"]  == 0) & (majorPartDf["gt"]  == 1) ]) / len(majorPartDf[ (majorPartDf["gt"]  == 1)])
fnr

0.3017241379310345

In [16]:
acc = len(majorPartDf[(majorPartDf["pred"]  == majorPartDf["gt"])]) / len(majorPartDf)
acc

0.8467432950191571

In [17]:
allSubsetAcc = []
for caseId in majorPartDf["CaseID"].unique().tolist():
    rows = majorPartDf[majorPartDf["CaseID"] == caseId]
    correct = rows[rows["pred"] == rows["gt"]]
    subset_acc = len(correct) / len(rows)
    allSubsetAcc.append(subset_acc)
    
    # print(subset_acc)
print(np.mean(allSubsetAcc))

0.846743295019157


In [18]:
caseStudyInspectDf.to_csv("./case_study_result.csv")